In [ ]:
!tar -xvf QA-Polyp-Dataset-2015-train.tar

In [6]:
import numpy as np
import os
import cv2 as cv
from sklearn.model_selection import train_test_split
import PIL
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [7]:
x_dataset = []
y_dataset = []
train_path = 'QA-Polyp/train'

with open(os.path.join(train_path,'train.txt')) as f:
  for line in f.readlines():
    img, label = line.split(' ')
    y_dataset.append(label)
    image = cv.imread(os.path.join(train_path, img))
    res = cv.resize(image,(224, 224), interpolation = cv.INTER_CUBIC)
    x_dataset.append(res)

print(x_dataset[0].shape)
print(y_dataset[0])

(224, 224, 3)
0



In [8]:
x_array = np.asarray(x_dataset)
y_array = np.asarray(y_dataset)
print(x_array.shape, y_array.shape)

(1062, 224, 224, 3) (1062,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(x_array, y_array, test_size=0.3, random_state=42)
print(X_train.shape)

(743, 224, 224, 3)


In [10]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.resnet50.ResNet50(include_top=False,
                   input_shape=(224, 224, 3),
                   pooling='avg',classes=2,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

94781440/94765736 [==============================] - 0s 0us/step


In [11]:
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(1, activation='sigmoid'))

In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
 flatten_1 (Flatten)         (None, 2)                 0         
                                                                 
 dense_2 (Dense)             (None, 512)               1536      
                                                                 
 dense_3 (Dense)             (None, 2)                 1

In [12]:
# Hyperparameters
LR = 0.001
EPOCHS = 10

In [15]:
resnet_model.compile(optimizer=Adam(lr=LR),loss='binary_crossentropy',metrics=['accuracy'])

history = resnet_model.fit(x=X_train, 
                           y=y_train,
                           batch_size=32,
                           epochs=10)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


24/24 [==============================] - 15s 102ms/step - loss: 0.2183 - accuracy: 0.9260
Epoch 2/10
24/24 [==============================] - 2s 88ms/step - loss: 0.1083 - accuracy: 0.9502
Epoch 3/10
24/24 [==============================] - 2s 86ms/step - loss: 0.0860 - accuracy: 0.9610
Epoch 4/10
24/24 [==============================] - 2s 85ms/step - loss: 0.0627 - accuracy: 0.9744
Epoch 5/10
24/24 [==============================] - 2s 85ms/step - loss: 0.0533 - accuracy: 0.9771
Epoch 6/10
24/24 [==============================] - 2s 85ms/step - loss: 0.0482 - accuracy: 0.9798
Epoch 7/10
24/24 [==============================] - 2s 85ms/step - loss: 0.0471 - accuracy: 0.9758
Epoch 8/10
24/24 [==============================] - 2s 85ms/step - loss: 0.0417 - accuracy: 0.9825
Epoch 9/10
24/24 [==============================] - 2s 86ms/step - loss: 0.0422 - accuracy: 0.9798
Epoch 10/10
24/24 [==============================] - 2s 85ms/step - loss: 0.0239 - accuracy: 0.9919


In [14]:
y_train = np.int_(y_train)
y_train.dtype

dtype('int64')

In [18]:
loss, acc = resnet_model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)

Test Accuracy: 0.947


In [17]:
y_test = np.int_(y_test)
y_test.dtype

dtype('int64')